In [1]:
from tqdm import tqdm
import pdb
import limix
import pickle as pkl
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from numpy import asarray
import xarray as xr
from copy import deepcopy
import os
from os.path import join
import simplejson as json
import hashlib
import pandas as pd
import pdb
from glob import glob
import h5py
import re
from plot import plot_poisson_ordered, plot_distplots, plot_distplot
from util import get_amelie_selection
import h5py
import dask
import dask.dataframe
import dask.array
from urllib.request import urlopen
try:
    import ipfsapi
    has_ipfs = True
except ModuleNotFoundError:
    has_ipfs = False


if has_ipfs:
    try:
        ipfs = ipfsapi.connect('127.0.0.1', 5001)
    except Exception:
        has_ipfs = False

only_hash = True

/nfs/software/stegle/users/horta/conda-envs/rats/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data (Amelie's and Hannah's)

In [2]:
if os.path.exists("data/traits-kinship-hannah-amelie.pkl"):
    data = pkl.load(open("data/traits-kinship-hannah-amelie.pkl", "rb"))
else:
    url = "http://ipfs.io/ipfs/QmXtFuoA3JTkhhpUpqh4dyu3FJLUG7DWAmXwUk1N2TMbAh"
    data = pkl.load(urlopen(url))
    
dst_folder = "3"
if not os.path.exists(dst_folder):
    os.mkdir(dst_folder)
if not os.path.exists(join(dst_folder, 'phenotype')):
    os.mkdir(join(dst_folder, 'phenotype'))
if not os.path.exists(join(dst_folder, 'kinship')):
    os.mkdir(join(dst_folder, 'kinship'))

In [3]:
def isbernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    if len(u) == 2:
        return True
    return False

def get_bernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    i0 = x == u[0]
    i1 = x == u[1]
    x[i0] = 0.0
    x[i1] = 1.0
    return x

def isdiscrete(x):
    x = np.asarray(x, float)
    ok = np.isfinite(x)
    return all(x[ok] == np.asarray(x[ok], int))

def get_poisson(x):
    x = np.asarray(x, float)
    mi = min(x)    
    if mi < 0:
        x += -mi
    return x

def isnumber(x):
    try:
        np.asarray(x, float)
    except ValueError:
        return False
    return True

In [4]:
def analysis_QTL(G, pheno, kinship, pheno_name, lik, pheno_norm):
    common_samples = set(kinship.index.values).intersection(set(pheno.index.values))
    common_samples = np.asarray(list(common_samples))
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))

    ok = np.isfinite(pheno.loc[:, pheno_name])
    common_samples = common_samples[ok]
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))
    
    if not all(np.isfinite(pheno[pheno_name])):
        raise ValueError("not all finite: {}".format(pheno_name))
    
    y = pheno_norm(pheno.loc[:, pheno_name])
    y = pd.Series(y, index=pheno.loc[:, pheno_name].index)
    
#     phenotype_series = pd.Series(y, pheno.loc[:, pheno_name].index.values)
    obj = pkl.dumps(y)
    m = hashlib.sha1()
    m.update(obj)
    phenotype_hex = m.hexdigest()
    with open(join(dst_folder, "phenotype", phenotype_hex + ".series.pkl"), "wb") as f:
        f.write(obj)
    
    obj = pkl.dumps(kinship)
    m = hashlib.sha1()
    m.update(obj)
    kinship_hex = m.hexdigest()
    with open(join(dst_folder, "kinship", kinship_hex + ".dataframe.pkl"), "wb") as f:
        f.write(obj)

    try:
        call = "limix.qtl.scan(G, y, lik, kinship, verbose=True)"
        model = limix.qtl.scan(G, y, lik, kinship, verbose=True)
    except ValueError as e:
        print(e)
        print("Pheno name: {}".format(pheno_name))
        return None

    return dict(pv=list(asarray(model.variant_pvalues)), lik=lik, call=call, phenotype=phenotype_hex, kinship=kinship_hex,
                null_covariate_effsizes=list(asarray(model.null_covariate_effsizes)),
                variant_effsizes=list(asarray(model.variant_effsizes)), variant_effsizes_se=list(asarray(model.variant_effsizes_se)))

In [5]:
f = h5py.File("arrayexpress/HS.hdf5", "r")
samples = np.asarray(f["/imputed_genotypes/row_header/rat"].value)
samples = [i.decode() for i in samples]
f.close()

In [6]:
data0 = deepcopy(data)
remove = []
for name in data0['measures'].columns.values:
    if isnumber(data0['measures'].loc[:, name]) and isdiscrete(data0['measures'].loc[:, name]):
        data0['measures'].loc[:, name] = get_poisson(data0['measures'].loc[:, name])
    else:
        remove.append(name)
for name in remove:
    del data0['measures'][name]

In [7]:
Gs = []
for i in tqdm(range(1, 22)):
    G = xr.open_dataarray("arrayexpress/HS.hdf5", "/imputed_genotypes/chr{}".format(i),
                          chunks=1000)
    G = G.rename({G.dims[0]: "snps", G.dims[1]: "samples"})
    Gs.append(G)

100%|██████████| 21/21 [00:00<00:00, 53.35it/s]


In [8]:
G = xr.concat(Gs, dim="snps").T
G['samples'] = samples
G['snps'] = range(G.shape[1])

In [ ]:
for name in data0['measures'].columns.values:
    print("Processing: {}".format(name))
    dst_file = join(dst_folder, "scan_measures_normal_" + name + "_mean_standardize.json")
    if os.path.exists(dst_file) or os.path.exists(dst_file + ".failed"):
        continue
    r = analysis_QTL(G, data0['measures'], data0['kinship'], name, 'normal', limix.qc.mean_standardize)
    if r is None:
        open(dst_file + ".failed", "w").write("")
    else:
        json.dump(r, open(dst_file, "w"))

Processing: Boli_tot
Processing: Distance0_30
Processing: Distance10_15
Processing: Distance15_20
Processing: Distance20_25


============================= qtl analysis session starts ==============================

Normalising input... 

/nfs/software/stegle/users/horta/conda-envs/rats/lib/python3.6/site-packages/xarray/core/indexing.py:1168: PerformanceWarning: Slicing with an out-of-order index is generating 282 times more chunks
  return self.array[key]


done (6.15 seconds).


...

Scale: 1.22413
Delta: 0.5
Beta: [-0.]
Scale: 1.2241
Delta: 0.5
Beta: [-0.30915243]
Scale: 1.2241
Delta: 0.5
Beta: [-0.30915232]
Scale: 1.2241
Delta: 0.500001
Beta: [-0.30915209]
Scale: 1.2241
Delta: 0.500001
Beta: [-0.30915163]
Scale: 1.2241
Delta: 0.500002
Beta: [-0.30915072]
Scale: 1.22409
Delta: 0.500005
Beta: [-0.30914889]
Scale: 1.22409
Delta: 0.500009
Beta: [-0.30914523]
Scale: 1.22407
Delta: 0.500019
Beta: [-0.30913792]
Scale: 1.22404
Delta: 0.500038
Beta: [-0.30912329]


33it [00:00, 417.02it/s]

Scale: 1.22399
Delta: 0.500076
Beta: [-0.30909403]
Scale: 1.22388
Delta: 0.500153
Beta: [-0.30903552]
Scale: 1.22367
Delta: 0.500305
Beta: [-0.30891853]
Scale: 1.22324
Delta: 0.50061
Beta: [-0.30868462]
Scale: 1.22238
Delta: 0.501221
Beta: [-0.30821716]
Scale: 1.22067
Delta: 0.502441
Beta: [-0.30728366]
Scale: 1.21726
Delta: 0.504883
Beta: [-0.30542232]
Scale: 1.21053
Delta: 0.509765
Beta: [-0.30172214]
Scale: 1.19741
Delta: 0.519522
Beta: [-0.29441127]
Scale: 1.17245
Delta: 0.538985
Beta: [-0.28014247]
Scale: 1.12741
Delta: 0.5775
Beta: [-0.25297726]
Scale: 1.05477
Delta: 0.651363
Beta: [-0.20382433]
Scale: 0.964133
Delta: 0.777312
Beta: [-0.12383683]
Scale: 0.910208
Delta: 0.924152
Beta: [-0.02047865]
Scale: 0.929212
Delta: 0.849099
Beta: [0.05604233]
Scale: 0.917638
Delta: 0.883155
Beta: [0.02312381]
Scale: 0.931343
Delta: 0.84379
Beta: [0.03992823]
Scale: 0.932111
Delta: 0.841922
Beta: [0.02026897]
Scale: 0.932011
Delta: 0.842165
Beta: [0.0192498]
Scale: 0.932008
Delta: 0.842171
Be


Scanning: 100%|██████████| 30/30 [43:45<00:00, 87.51s/it]


===================== qtl analysis session ends in 2637.86 seconds =====================

In [ ]:
# for name in data0['measures'].columns.values:
#     print("Processing: {}".format(name))
#     dst_file = join(dst_folder, "scan_measures_normal_" + name + "_quantile_gaussianize.json")
#     if os.path.exists(dst_file) or os.path.exists(dst_file + ".failed"):
#         continue
#     r = analysis_QTL(G, data0['measures'], data0['kinship'], name, 'normal', limix.qc.quantile_gaussianize)
#     if r is None:
#         open(dst_file + ".failed", "w").write("")
#     else:
#         json.dump(r, open(dst_file, "w"))

In [ ]:
f.close()